In [1]:

from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.applications import VGG16
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Flatten, Dropout, Activation, Input, Conv2D, MaxPooling2D
from keras.models import Model
from keras import Sequential
from sklearn.preprocessing import LabelBinarizer
import numpy as np
import pickle
import os

In [2]:
learning_rate = 0.01 #diminui o valor do learning rate pois estamos usando Adam como otimizador

## Treinamento Rede VGG

In [3]:
def get_images(pasta):
    exts = ['.PNG','.JPG','.JPEG','.TIFF','.GIF','.BMP']
    tot_images = 0
    for sf in [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))]:
        subdir = os.path.join(pasta,sf)
        tot_images = tot_images + len([name for name in os.listdir(subdir) if os.path.splitext(name)[1].upper() in exts])
    return tot_images

In [4]:
# FUNÇÃO PARA LER O NOME DAS CLASSES
def get_labels(pasta):
    return [name for name in os.listdir(pasta) if os.path.isdir(os.path.join(pasta, name))];


In [5]:
EPOCHS = 50;
BATCH_SIZE = 16; #serão gerados batches de 16 imagens (a rede vê 16 imagens de cada vez durante o treino). Esse número é para facilitar o processamento conforme a memória do computador durante o treino, colocamos 16 não travar
IMG_SIZE = (224,224,3);
optimizer_name = 'SGD'

In [6]:
trainFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\train'
valFolder = 'C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\db\\val'
labels = get_labels(trainFolder)
labels = np.array(labels);

# Organiza os labels em matriz e salva, para posterior uso em classificação
lb = LabelBinarizer();
labels = lb.fit_transform(labels);
f = open('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\budioes_vgg_' + optimizer_name + '_' + str(learning_rate) + ".pickle", "wb")
f.write(pickle.dumps(lb));
f.close();

ImageDataGenerator = Carrega aos poucos as imagens em memória para fazer a leitura (para não estourar a memória)

In [7]:
#ImageDataGenerator = objeto para buscar as imagens em uma pasta
#Treino
augTrain = ImageDataGenerator(rotation_range=20, width_shift_range = 0.1, height_shift_range = 0.1, 
                              shear_range = 0.15, zoom_range = [1.0, 1.25], horizontal_flip=True, 
                              fill_mode="nearest");
#Validação
augVal = ImageDataGenerator();

Relu = 

Maxpooling = selecionar o pixel com maior valor em uma dimensão 2D de 2x2 (no exemplo abaixo, poderia ser 3x3, 4x4 etc). Nesse caso, com uma janela 2x2, o maxpooling vai reduzir o tamanho da imagem pela metade.

In [8]:
#ABAIXO TEMOS UMA REDE SIMPLES. ELA É NA VERDADE "A BASE" DE TODAS AS REDES
#PARA EXEMPLIFICAR, VAMOS DIVIDIR AS ETAPAS DA REDE VGG

#REDE CONVOLUCIONAL:

#model = Sequential()
#32 = camadas da imagem e 5x5 é o tamanho da janela de varredura
#model.add(Conv2D(32, (5, 5), activation='relu'))
#maxpooling reduz o tamanho da imagem, 2x2 irá reduzir a imagem pela metade (vai pular 2 pixels a cada "janelamento")
#model.add(MaxPooling2D((2, 2))) 
#Reduzir a janela de 5x5 para 3x3
#model.add(Conv2D(64, (3, 3), activation='relu'))
#model.add(MaxPooling2D((2, 2)))
#model.add(Conv2D(128, (3, 3), activation='relu'))
#model.add(MaxPooling2D((2, 2)))
#model.add(Conv2D(256, (3, 3), activation='relu'))
#model.add(MaxPooling2D((2, 2)))
#Transforma o "cubo" em 1 vetor e 1 dimensão (vai receber um cubo de 28x28x8 e transformar em um vetor de 6272)

#O PARÂMETRO INCLUDE_TOP = TRUE FAZ O BLOCO DE CÓDIGO ABAIXO, A PARTE "FINAL" DA REDE, A REDE NEURAL (NÃO CONVOLUCIONAL)
#NO NOSSO EXEMPLO, O INCLUDE_TOP=FALSE. Significa que vamos fazer "na mão" essa etapa e desconsiderar o padrão do VGG

#model.add(Flatten())
#Camadas da rede neural:
#Vai receber um vetor de 6272 e usar de input nas camadas da rede
#Por isso temos que fazer várias camadas, para não "estrangular" a rede e obter o resultado final binário
#model.add(Dense(512, activation='relu'))
#model.add(Dense(256, activation='relu'))
#Resultado final da rede. No caso dos budiões, estamos trabalhando com 3 resultados possíveis. Por isso é 3.
#model.add(Dense(3, activation='sigmoid'))

In [9]:
#Declaração da rede VGG
# Pega só as camadas de convolução e retreina as de classificação

baseModel = VGG16(include_top=False, weights="imagenet", input_tensor=Input(shape=(224, 224, 3)))

In [10]:
#Como vamos usar imagenet, não faz sentido treinar a rede novamente pois já vamos usar o modelo treinado
for layer in baseModel.layers:
    layer.trainable = False

In [11]:
#Include_Top feito "Na mão":

#É preciso criar novas camadas (headModel) pra zerar os pesos da VGG
headModel = baseModel.output
headModel = Flatten(name='flatten')(headModel) #(headModel) é o mesmo que concatenar usando o .add
headModel = Dense(4096,  name='fc1')(headModel)
headModel = Activation('relu',  name='act_fc1')(headModel)
#Recurso para evitar overfit (regularização), no caso, 20% dos parâmetros são zeros a cada iteração da rede. 
#Isso força a rede a aprender outras formas de classificar
headModel = Dropout(0.2) (headModel)
headModel = Dense(2048,  name='fc2')(headModel)
headModel = Activation('relu',  name='act_fc2')(headModel)
headModel = Dropout(0.2) (headModel)
headModel = Dense(512,  name='fc3')(headModel)
headModel = Activation('relu',  name='act_fc3')(headModel)
headModel = Dropout(0.2) (headModel)
headModel = Dense(len(lb.classes_), activation="softmax", name='predictions')(headModel)
#Junta tudo num modelo só
model = Model(inputs=baseModel.input, outputs=headModel)

In [12]:
#model.summary()

In [13]:
if optimizer_name == 'Adam':
    optimizer = Adam(learning_rate=learning_rate)

if optimizer_name == 'SGD':
    optimizer = SGD(learning_rate=learning_rate)

In [14]:
#No nosso modelo, o callback irá salvar o melhor modelo entre as épocas (epochs) - função Model Checkpoint
callbacks = [
    #ReduceLROnPlateau(monitor = 'val_acc',factor=0.85, patience=10, min_lr=0.000001, verbose=1),
    ModelCheckpoint('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_vgg_' + optimizer_name + '_' + str(learning_rate) + 
                    '-ckpnt.model', save_best_only=True, monitor='val_accuracy', mode='max', verbose = 1)
]  

In [15]:
#Reservando espaço de memória para a rede funcionar
#Ao final de cada época, será rodado o comando "callbacks"
model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=['accuracy'])

In [16]:
#Função para buscar imagens
#TREINO
trainGen = augTrain.flow_from_directory(
    trainFolder, #caminho da imagem
    class_mode="categorical", #o nome da pasta onde está a imagem será o nome da classe
    target_size=(IMG_SIZE[0], IMG_SIZE[1]), #tamanho da imagem a ser redimensionada
    color_mode="rgb", #a imagem terá 3 canais RGB
    shuffle=True, #vai embaralhar as imagens enquanto faz a leitura
    batch_size=BATCH_SIZE); #de quantas em quantas imagens será feita a leitura (tamanho do BATCH)

Found 485 images belonging to 3 classes.


In [17]:
#Função para buscar imagens
#VALIDAÇÃO
valGen = augVal.flow_from_directory(
    valFolder,
    class_mode="categorical",
    target_size=(IMG_SIZE[0], IMG_SIZE[1]),
    color_mode="rgb",
    shuffle=True,
    batch_size=BATCH_SIZE);

Found 122 images belonging to 3 classes.


In [18]:
#fit_generator vai de fato treinar a rede
trained_model = model.fit(trainGen, validation_data=valGen,
                        steps_per_epoch=get_images(trainFolder)//BATCH_SIZE,
                        validation_steps=get_images(valFolder) // BATCH_SIZE,
                        epochs = EPOCHS, callbacks=callbacks, verbose =1);
#Dividir o número de imagens pelo número de batchs para garantir que cada BATCH seja lido a cada época
#tanto no treino (steps_per_epoch) quanto na validação (validation_steps)
#verbose = dá output da rede a cada final de época


Epoch 1/50
30/30 [==============================] - 14s 308ms/step - loss: nan - accuracy: 0.3134 - val_loss: nan - val_accuracy: 0.3214

Epoch 00001: val_accuracy improved from -inf to 0.32143, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_vgg_SGD_0.01-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_vgg_SGD_0.01-ckpnt.model\assets
Epoch 2/50
30/30 [==============================] - 8s 248ms/step - loss: nan - accuracy: 0.3070 - val_loss: nan - val_accuracy: 0.3125

Epoch 00002: val_accuracy did not improve from 0.32143
Epoch 3/50
30/30 [==============================] - 8s 254ms/step - loss: nan - accuracy: 0.3028 - val_loss: nan - val_accuracy: 0.3304

Epoch 00003: val_accuracy improved from 0.32143 to 0.33036, saving model to C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_vgg_SGD_0.01-ckpnt.model
INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos

30/30 [==============================] - 8s 258ms/step - loss: nan - accuracy: 0.3028 - val_loss: nan - val_accuracy: 0.3125

Epoch 00040: val_accuracy did not improve from 0.33929
Epoch 41/50
30/30 [==============================] - 8s 264ms/step - loss: nan - accuracy: 0.3006 - val_loss: nan - val_accuracy: 0.3036

Epoch 00041: val_accuracy did not improve from 0.33929
Epoch 42/50
30/30 [==============================] - 8s 255ms/step - loss: nan - accuracy: 0.3049 - val_loss: nan - val_accuracy: 0.2857

Epoch 00042: val_accuracy did not improve from 0.33929
Epoch 43/50
30/30 [==============================] - 8s 263ms/step - loss: nan - accuracy: 0.3113 - val_loss: nan - val_accuracy: 0.2857

Epoch 00043: val_accuracy did not improve from 0.33929
Epoch 44/50
30/30 [==============================] - 8s 250ms/step - loss: nan - accuracy: 0.3070 - val_loss: nan - val_accuracy: 0.2857

Epoch 00044: val_accuracy did not improve from 0.33929
Epoch 45/50
30/30 [============================

In [19]:
#Salva o modelo (pesos + conexões entre os neurônios, ou seja, a estrutura da rede)
model.save('C:\\Users\\LDT\\Desktop\\mestrado-unifesp\\exemplos\\modelo_budioes_vgg_' + optimizer_name + '_' + str(learning_rate) + ".model");

INFO:tensorflow:Assets written to: C:\Users\LDT\Desktop\mestrado-unifesp\exemplos\modelo_budioes_vgg_SGD_0.01.model\assets


In [20]:
#https://www.learndatasci.com/tutorials/hands-on-transfer-learning-keras/
#https://neptune.ai/blog/keras-metrics